# How to data parallel
- intro to qml
- training with GPU
- training with multi GPU
- training with multi instances
- input output
- save and load model
- eval accuracy

## Quantum machine learning
Quantum machine learning (QML) is a special type of hybrid quantum-classical workload. Like classical machine learning (ML), there are usually a parameterized model, a dataset and a loss funcion. When training the model with the dataset, the parameters of the model are updated to minimize the loss funcion. In QML, the model contains one or many quantum circuits. The model may or may not also inlcude classical neural nets. A loss functions is usually defined for a single data point. Say a dataset $D$ has $N_D$ data points, $d_1$, $d_1$, ..., $d_{N_D}$. The losses associated with the data points are $L(d_1)$, $L(d_1)$, ..., $L(d_{N_D})$, where $L$ is the loss function. Without invoking any advanced feature, the algorithm script would compute these losses in serial, and then average them to be the total loss for gradient computations. This procedure is time consuming, especially when there are hundreds of data points. 

## Data parallelism
The loss from one data point is independent to the other data points. The order of the loss evaluations therefore do not need to follow a specific order. They can even be evaluated <i>in parallel</i>! Losses and gradients of variational parameters associated with different data points can be evaluated on different GPU cores at the same time. This is known data parallelism. In this notebook, we will learn to use [SageMaker's distributed data parallel library](https://docs.aws.amazon.com/sagemaker/latest/dg/data-parallel.html) to accerlate the training of your quantum model. We go through examples to show you how to parallelize trainings across multiple GPUs in an instance, and even multiple GPUs over multiple instances! 

## Binary Classification of Sonar dataset
Let's use a binary classification of the [Sonar dataset](https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+%28Sonar%2C+Mines+vs.+Rocks%29) as the QML example. The Sonar dataset contains 208 data points each with 60 features that are collected from sonar signals bouncing off materials. Each data points is either labeled as "M" for mines or "R" for rocks. Our QML model consists of an input layer, a quantum circuit and an output layer. The input and output layer are classical dense layer. The dimension of classical input layer is $60\times N$, where $N$ is the number of qubit in the quantum circuit. The result of the input layer is encoded into the quantum circuit using [angle embedding](https://pennylane.readthedocs.io/en/stable/code/api/pennylane.AngleEmbedding.html). After the angle embedding, the quantum circuit has the same structure as figure 4 of [this paper](https://arxiv.org/abs/1804.00633). It is a generic circuit ansatz that has two parametrized [strongly entangle layers](https://pennylane.readthedocs.io/en/user-docs-refactor/code/pennylane.templates.layers.StronglyEntanglingLayer.html) and a single parametrized rotation gate at the first qubit. The measurement is only performed at the first qubit. Using the concept of classical ML, the dimension of the quantum circuit layer is $N\times1$. The classical output layer has dimension $1\times1$ which takes the measurment of the quantum circuit and outputs a real number. The loss function is the [margin loss function](https://pytorch.org/docs/stable/generated/torch.nn.MarginRankingLoss.html) between the model output and the label. See [model_def.py](source_script/model_def.py) and [quantum_circuit.py](source_script/quantum_circuit.py) for more detail about the model and the quantum circuits.

In [1]:
input_file_path = "data/sonar.all-data"

## Training with single GPU core

Let's start by running the job with `lightning.gpu` simulator on a single GPU core in a `ml.p3.2xlarge` instance which has one Nvidia V100 GPU core. The algorithm script to train our quanutm model is [train_single.py](source_script/train_single.py). In the algorithm script, we use Pennylane with PyTorch as our framework, which are both included in Braket's pre-configured PyTorch container. 

In [2]:
from braket.jobs.config import InstanceConfig

instance_config = InstanceConfig(instanceType='ml.p3.2xlarge')

hyperparameters={"nwires": "10", 
                 "ndata": "16", 
                 "batch_size": "16", 
                 "epochs": "5", 
                 "gamma": "0.99", 
                 "lr": "0.01",
                 "seed": "164",
                 "to_eval": "0",
                }

input_file_path = "data/sonar.all-data"

image_uri="537332306153.dkr.ecr.us-west-2.amazonaws.com/lightning-bjob:v9" # Remove this line after public container builds

We submit our job after setting up instance configuration, hyperparameters and the job container image.

In [3]:
import time
from braket.aws import AwsQuantumJob

job = AwsQuantumJob.create(
    device="local:pennylane/lightning.gpu",
    source_module="source_script",
    entry_point="source_script.train_single",
    job_name="dp-" + str(int(time.time())),
    hyperparameters=hyperparameters,
    input_data={"input-data": input_file_path},
    instance_config=instance_config,
    image_uri=image_uri,
    wait_until_complete=False,
)

In [4]:
# This cell should take about 6 minutes
print(job.result())

{'last loss': 0.09291739761829376}


## Modify your algorithm script for data parallelism <a class="anchor" id="modify"></a>

[PyTorch](https://pytorch.org/tutorials/intermediate/ddp_tutorial.html) and [TensorFlow](https://www.tensorflow.org/guide/distributed_training) have built-in features for data parallelism. With SageMaker's distributed data parallel library, Braket Jobs makes it easier for you to leverage data parallelism to accelerate your training. In this notebook we use apply data parallelism with PyTorch. To use data parallelism, you need to slightly modified your algorithm script. As an example, we modify the algorithm script [train_single.py](source_script/train_single.py) to [train_dp.py](source_script/train_dp.py). Let's go through the changes.

First, we import the `smdistributed` package which does most of the heavylifting for distributing your workloads accross multiple GPU and/or multiple instances. This package is pre-configured in the Braket PyTorch and TensorFlow containers. The `DDP` function from `smdistributed` converts the quantum model into a data parallelizable model. The `dist` module  tell our algorithm script the total number of instances for the training (`world_size`), and the `rank` and `local_rank` of a GPU core. `rank` is the absolute index of a GPU across all instances, while `local_rank` is the index of a GPU within an instance. For example, if there are four instances each with eight GPUs allocated for the trainin, the `rank` ranges from 0 to 31 and the `local_rank` ranges from 0 to 7.



In [25]:
!sed -n 23,24p source_script/train_dp.py
!sed -n 65,72p source_script/train_dp.py

import smdistributed.dataparallel.torch.distributed as dist
from smdistributed.dataparallel.torch.parallel.distributed import DistributedDataParallel as DDP
    dp_info = {
        "world_size": dist.get_world_size(),
        "rank": dist.get_rank(),
        "local_rank": dist.get_local_rank(),
    }
    batch_size //= dp_info["world_size"] // 8
    batch_size = max(batch_size, 1)
    print("dp_info: ", dp_info)


Next, we define a `DistributedSampler` according to the `world_size` and `rank`, and pass it into the data loader. This sampler avoids GPUs accessing the same slice of a dataset. 

In [18]:
!sed -n 81,93p source_script/train_dp.py

    train_sampler = torch.utils.data.distributed.DistributedSampler(
        train_dataset, 
        num_replicas=dp_info["world_size"], 
        rank=dp_info["rank"]
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        sampler=train_sampler,        
    )


Next, we use the `DDP` fuction to make our quantum model parallelizable. 

In [24]:
!sed -n 105,108p source_script/train_dp.py

    model = DressedQNN(qc_dev).to(device)
    model = DDP(model)
    torch.cuda.set_device(dp_info["local_rank"])
    model.cuda(dp_info["local_rank"])


The above are the programming change you need to make to use data parallelism. In QML, it is very often that we want to save models, save results and print training progress. If each GPU executes the saving and printint command, the log would be flooded with the repeated information, and the model and results would be overwrting wach other. To avoid this, we only save and print from the GPU that has `rank` 0. 

In [20]:
!sed -n 134,137p source_script/train_dp.py

    if dp_info["rank"]==0:    
        print('elapsed time: ', elapsed)
        torch.save(model.state_dict(), f"{output_dir}/test_local.pt")
        save_job_result({"last loss": loss_before})


## Training with multiple GPU cores in single instance
With the modified algorithm, we can now submit our job with data parallelism. The SageMaker distributed data parallel library only supports three instance types listed below. Be sure to choose the instance type from the list and configure it through the `InstanceConfig` argument in Jobs. 
- ml.p3.16xlarge
- ml.p3dn.24xlarge
- ml.p4d.24xlarge

For the SageMaker distributed data parallel library to know that data parallelism is enabled, we provide two additional hyperparameters for the library. `"sagemaker_distributed_dataparallel_enabled"` is set to `"true"`, and `"sagemaker_instance_type"` needs to be the instance type we are using. Keep in mind that data parallelism only works correctly when you modify your algorithm script according to the [previous section](#modify). If the data parallelism option is enabled in the hyperparameters without a correctly modified algorithm script, the job may throws errors, or each GPU may repeat the same workload without data parallelism.

In [11]:
from braket.jobs.config import InstanceConfig

instance_config = InstanceConfig(instanceType='ml.p3.16xlarge')

hyperparameters={"nwires": "10", 
                 "ndata": "32", 
                 "batch_size": "32", 
                 "epochs": "5", 
                 "gamma": "0.99", 
                 "lr": "0.01",
                 "seed": "164",
                 "to_eval": "0",
                 "sagemaker_distributed_dataparallel_enabled": "true",
                 "sagemaker_instance_type": "ml.p3.16xlarge",
                }

input_file_path = "data/sonar.all-data"

image_uri="537332306153.dkr.ecr.us-west-2.amazonaws.com/lightning-bjob:v9" # Remove this line after public container builds

When the instance and data parallelism configured, we can now submit our job. There are 8 GPUs in a `ml.p3.16xlarge` instance. When the instance spins up, the workload is distributed across the 8 GPUs.

In [12]:
import time
from braket.aws import AwsQuantumJob

job = AwsQuantumJob.create(
    device="local:pennylane/lightning.gpu",
    source_module="source_script",
#     entry_point="source_script.train_dp_test",
    entry_point="source_script.train_dp",
    job_name="dp-" + str(int(time.time())),
    hyperparameters=hyperparameters,
    input_data={"input-data": input_file_path},
    instance_config=instance_config,
    image_uri=image_uri,
    wait_until_complete=False,
)

In [ ]:
# This cell should take about 6 minutes
print(job.result())

## Training with multiple GPU cores across multiple instances

We are not limited to parallelize the workload inisde a single instances. We can perform distributed training by parallizing our workload across multiple instances. With the algorithm script modified and the data parallism enabled, we can perform distributed data parallelism by setting instance count larger than 1. To configure instance count, we use the `instanceCount` argument in `InstanceConfig`. The SageMaker distributed library we included in our algorithm script coordinates the multiple instances and conducts the distributed training for us. 

In [27]:
instance_config = InstanceConfig(instanceType='ml.p3.16xlarge', instanceCount=2)

Be mindful that, when using multiple instances, each instance incurs charge based on how much time you use it. In distributed data parallelism, when you use set `instanceCount=2`, two instances are allocated to run your job. SageMaker distributed library managed the instances that they start and end at the same time. If your workload takes 200 seconds, you will be billed for 200 seconds for each instance used which adds to 400 seconds in total.

In [ ]:
from braket.jobs.config import InstanceConfig

instance_config = InstanceConfig(instanceType='ml.p3.16xlarge')

hyperparameters={"nwires": "10", 
                 "ndata": "32", 
                 "batch_size": "32", 
                 "epochs": "5", 
                 "gamma": "0.99", 
                 "lr": "0.01",
                 "seed": "164",
                 "to_eval": "0",
                 "sagemaker_distributed_dataparallel_enabled": "true",
                 "sagemaker_instance_type": "ml.p3.16xlarge",
                }

input_file_path = "data/sonar.all-data"

image_uri="537332306153.dkr.ecr.us-west-2.amazonaws.com/lightning-bjob:v9" # Remove this line after public container builds

Now we can submit our job for distributed data parallelism!

In [28]:
import time
from braket.aws import AwsQuantumJob

job = AwsQuantumJob.create(
    device="local:pennylane/lightning.gpu",
    source_module="source_script",
    entry_point="source_script.train_dp",
    job_name="dp-" + str(int(time.time())),
    hyperparameters=hyperparameters,
    input_data={"input-data": input_file_path},
    instance_config=instance_config,
    image_uri=image_uri,
    wait_until_complete=False,
)

In [ ]:
# This cell should take about 6 minutes
print(job.result())

## Debug with local mode

In [7]:
hyperparameters={"nwires": "10", 
                 "ndata": "16", 
                 "batch_size": "16", 
                 "epochs": "5", 
                 "gamma": "0.99", 
                 "lr": "0.03",
                 "seed": "164",
                 "to_eval": "0",
                }
input_file_path = "data/sonar.all-data"

In [8]:
import time
from braket.jobs.local.local_job import LocalQuantumJob

# This cell should take about 2 min for the fisrt time, and about 30 seconds afterward.
job = LocalQuantumJob.create(
    device="local:pennylane/lightning.qubit",
    source_module="source_script",
    entry_point="source_script.train_single",
    job_name="dp-" + str(int(time.time())),
    hyperparameters=hyperparameters,
    input_data={"input-data": input_file_path},
    image_uri=image_uri,
)

Using the short-lived AWS credentials found in session. They might expire while running.


Boto3 Version:  1.20.10
Beginning Setup
Running Code As Subprocess
hyperparams:  {'nwires': '10', 'ndata': '16', 'batch_size': '16', 'epochs': '5', 'gamma': '0.99', 'lr': '0.03', 'seed': '164', 'to_eval': '0'}
Using local simulator:  Lightning Qubit PennyLane plugin
[2022-04-15 20:33:24.097 bf620bd46683:51 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-04-15 20:33:24.132 bf620bd46683:51 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Train Epoch: 1 [0/16 (0%)]	Loss: 0.191733
Metrics - timestamp=1650054804.4977372; Loss=0.19173291325569153; iteration_number=1;
Train Epoch: 2 [0/16 (0%)]	Loss: 0.200744
Metrics - timestamp=1650054804.7766159; Loss=0.20074376463890076; iteration_number=2;
Train Epoch: 3 [0/16 (0%)]	Loss: 0.120848
Metrics - timestamp=1650054805.1639974; Loss=0.12084810435771942; iteration_number=3;
Train Epoch: 4 [0/16 (0%)]	Loss: 0.102050
Metrics - timestamp=1650054805.4387467; Loss=0

In [9]:
print(job.result())

{'last loss': 0.10789139568805695}


## Summary
In this notebook we showed you how to get started with running simulators embedded in Hybrid Jobs. To learn more, you can read the [documentation](https://docs.aws.amazon.com/braket/latest/developerguide/braket-jobs.html) or follow the other example notebooks in this repo.

In [ ]:
# !pip uninstall --yes amazon-braket-sdk 

In [ ]:
# !pip install git+https://github.com/aws/amazon-braket-sdk-python.git@local-sim-jobs

In [7]:
# !aws configure add-model --service-model "file://braket-2019-09-01.normal.json" --service-name braket